In [1]:
import ray
from NameNode import NameNode
from StorageNode import StorageNode
from Chunk import Chunks
import Config
from Store import *
from Update import *
from Remove import *
from Get import *
import os

In [ ]:
if ray.is_initialized():
    ray.shutdown()
# 192.168.0.3 to telefon z androidem
ray.init(address="ray://192.168.238.170:10001", runtime_env={
    "working_dir": os.getcwd()
})

2025-05-13 17:11:21,177	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
2025-05-13 17:11:21,530	INFO packaging.py:575 -- Creating a file package for local module 'c:\studia\rozprochy\ray'.
2025-05-13 17:11:21,574	INFO packaging.py:367 -- Pushing file package 'gcs://_ray_pkg_e1bcd06d0912b6e6.zip' (0.06MiB) to Ray cluster...
2025-05-13 17:11:21,606	INFO packaging.py:380 -- Successfully pushed file package 'gcs://_ray_pkg_e1bcd06d0912b6e6.zip'.
SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.12.9
Ray version:,2.44.1


(NameNode pid=26414) new node: StorageNode0, @ 0
(NameNode pid=26414) new node: StorageNode1, @ 1
(NameNode pid=26414) new node: StorageNode2, @ 2
(NameNode pid=26414) new node: StorageNode3, @ 3
(NameNode pid=26414) storage(): "data" not in locations
(NameNode pid=26414) stored data "data" chunk 0 in 3
(3 pid=26474) {'data': [0]}
(NameNode pid=26414) stored data "data" chunk 0 in 2
(2 pid=26475) {'data': [0]}
(0 pid=26479) {'data': [0]}
(NameNode pid=26414) stored data "data" chunk 0 in 0
(NameNode pid=26414) stored data "data" chunk 1 in 3
(NameNode pid=26414) stored data "data" chunk 1 in 2
(NameNode pid=26414) stored data "data" chunk 1 in 1
(1 pid=26478) {'data': [1]}
(2 pid=26475) {'data': [0, 1]}
(3 pid=26474) {'data': [0, 1]}
(3 pid=26474) {'data': [0, 1]}
(2 pid=26475) {'data': [0, 1]}
(0 pid=26479) {'data': [0]}
(3 pid=26474) {'data': [0, 1]}
(2 pid=26475) {'data': [0, 1]}
(1 pid=26478) {'data': [1]}
(NameNode pid=26414) stored data "data" chunk 2 in 1
(1 pid=26478) {'data': 

In [3]:
nn = NameNode.remote()

In [4]:
name = 'data'
data = 'Lorem ipsum dolor sit amet, consectetur'
len(data)

39

In [5]:
# pobieram storage na dane (propozycja)
storageForData = ray.get(nn.storage.remote(name, Chunks.count(len(data))))
for chunk in range(Chunks.count(len(data))):
    print(f'Chunk {chunk} can be stored in nodes:')
    print(f'\t{list(map(lambda x: x[0], storageForData[chunk]))}')

Chunk 0 can be stored in nodes:
	[3, 2, 0]
Chunk 1 can be stored in nodes:
	[1, 3, 2]


In [6]:
# zapisuję dane w proponowanych node'ach
store(name, storageForData, data)

In [7]:
print(ray.get(nn.state.remote()))

{'data': [[3, 2, 0], [3, 2, 1]]}
[{'data': {0}}, {'data': {1}}, {'data': {0, 1}}, {'data': {0, 1}}]
{0: {'data': [0]}, 1: {'data': [1]}, 2: {'data': [0, 1]}, 3: {'data': [0, 1]}}


In [ ]:
# odczyt kolejncyh chunków
nodes = [storageForData[i][0] for i in range(Chunks.count(len(data)))]
print(ray.get(nn.state.remote()))
for i, pair in enumerate(nodes):
    print(f'{i}: \'{ray.get(pair[1].get.remote(name, i))}\'')

In [8]:
dataMore = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad'

In [9]:
# pobieram storage dla update (propozycja)
storageUpdate = ray.get(nn.updateStorage.remote(name, Chunks.count(len(dataMore))))
storageUpdate

([[(3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (2, ClientActorHandle(3237ac34170829e4e3c117d401000000)),
   (0, ClientActorHandle(ff8580e22acd09b1f30afbc401000000))],
  [(3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (2, ClientActorHandle(3237ac34170829e4e3c117d401000000)),
   (1, ClientActorHandle(c695684a5657b23d5ed6a26201000000))]],
 None,
 [[(1, ClientActorHandle(c695684a5657b23d5ed6a26201000000)),
   (3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (0, ClientActorHandle(ff8580e22acd09b1f30afbc401000000))],
  [(2, ClientActorHandle(3237ac34170829e4e3c117d401000000)),
   (3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (1, ClientActorHandle(c695684a5657b23d5ed6a26201000000))],
  [(3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (0, ClientActorHandle(ff8580e22acd09b1f30afbc401000000)),
   (1, ClientActorHandle(c695684a5657b23d5ed6a26201000000))]])

In [10]:
# aktualizuję wpis zgodnie z propozycją
update(name, storageUpdate, dataMore)

In [11]:
storageMore = ray.get(nn.storage.remote(name, Chunks.count(len(dataMore))))
print(ray.get(nn.state.remote()))
nodesMore = [storageMore[i][0] for i in range(Chunks.count(len(dataMore)))]
for i, pair in enumerate(nodesMore):
    print(f'{i}: \'{ray.get(pair[1].get.remote(name, i))}\'')

{'data': [[3, 2, 0], [3, 2, 1], [1, 3, 0], [2, 3, 1], [3, 0, 1]]}
[{'data': {0, 2, 4}}, {'data': {1, 2, 3, 4}}, {'data': {0, 1, 3}}, {'data': {0, 1, 2, 3, 4}}]
{0: {'data': [0, 2, 4]}, 1: {'data': [1, 2, 3, 4]}, 2: {'data': [0, 1, 3]}, 3: {'data': [0, 1, 2, 3, 4]}}
0: 'Lorem ipsum dolor sit amet, cons'
1: 'ectetur adipiscing elit, sed do '
2: 'eiusmod tempor incididunt ut lab'
3: 'ore et dolore magna aliqua. Ut e'
4: 'nim ad'


In [12]:
dataUpper = data.upper()
storageUpdateUpper = ray.get(nn.updateStorage.remote(name, Chunks.count(len(dataUpper))))
storageUpdateUpper

([[(3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (2, ClientActorHandle(3237ac34170829e4e3c117d401000000)),
   (0, ClientActorHandle(ff8580e22acd09b1f30afbc401000000))],
  [(3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (2, ClientActorHandle(3237ac34170829e4e3c117d401000000)),
   (1, ClientActorHandle(c695684a5657b23d5ed6a26201000000))]],
 [[(1, ClientActorHandle(c695684a5657b23d5ed6a26201000000)),
   (3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (0, ClientActorHandle(ff8580e22acd09b1f30afbc401000000))],
  [(2, ClientActorHandle(3237ac34170829e4e3c117d401000000)),
   (3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (1, ClientActorHandle(c695684a5657b23d5ed6a26201000000))],
  [(3, ClientActorHandle(03963ed8cd220454b350573001000000)),
   (0, ClientActorHandle(ff8580e22acd09b1f30afbc401000000)),
   (1, ClientActorHandle(c695684a5657b23d5ed6a26201000000))]],
 None)

In [13]:
update(name, storageUpdateUpper, dataUpper)

In [16]:
storageUpper = ray.get(nn.storage.remote(name, Chunks.count(len(dataUpper))))
print(ray.get(nn.state.remote()))
nodesMore = [storageUpper[i][0] for i in range(Chunks.count(len(dataUpper)))]
for i, pair in enumerate(nodesMore):
    print(f'{i}: \'{ray.get(pair[1].get.remote(name, i))}\'')

{'data': [[3, 2, 0], [3, 2, 1]]}
[{'data': {0}}, {'data': {1}}, {'data': {0, 1}}, {'data': {0, 1}}]
{0: {'data': [0]}, 1: {'data': [1]}, 2: {'data': [0, 1]}, 3: {'data': [0, 1]}}
0: 'LOREM IPSUM DOLOR SIT AMET, CONS'
1: 'ECTETUR'


In [19]:
print(ray.get(nn.state.remote()))

{}
[{}, {}, {}, {}]
{0: {}, 1: {}, 2: {}, 3: {}}


In [15]:
# usuwam aktora i naprawiam błędy
ray.kill(ray.get_actor("StorageNode0"))
ray.get(nn.assure.remote())

In [17]:
# pobieram artefakt
print(get(name, ray.get(nn.getStorage.remote(name))))

LOREM IPSUM DOLOR SIT AMET, CONSECTETUR


In [18]:
# usuwam artefakt
remove(name, ray.get(nn.removeStorage.remote(name)))